## อ่านไฟล์ DimReseller.csv เข้า dataframe

### แสดง 5 แถวข้อมูลแรกใน dataframe

In [18]:
df = spark.read.format("csv").option("header","true").load("Files/Source/DimReseller.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Source/DimReseller.csv".

display(df.limit(5))

StatementMeta(, 0050a1b7-1a65-4ae9-ace5-9e601e01c049, 20, Finished, Available)

SynapseWidget(Synapse.DataFrame, eedfce79-1fb9-4139-ac3d-182cb36ee1e5)

#XXXX

## สร้าง dataframe ใหม่ชื่อ reseller_df จาก dataframe เดิม

### โดยลบคอลัมน์ที่ไม่ใช้ออก

In [19]:
cols = ("ResellerAlternateKey","Phone","NumberEmployees","AddressLine1","AddressLine2","MinPaymentType","MinPaymentAmount","AnnualSales","AnnualRevenue",
        "OrderMonth","FirstOrderYear","LastOrderYear","YearOpened")

reseller_df = df.drop(*cols)

display(reseller_df.limit(5))

StatementMeta(, 0050a1b7-1a65-4ae9-ace5-9e601e01c049, 21, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8376db03-179c-4c98-a5d4-b7ef3a0ecb1f)

## อ่านไฟล์ DimGeography.csv เข้า dataframe

### แสดง 5 แถวข้อมูลแรกใน dataframe

In [20]:
df = spark.read.format("csv").option("header","true").load("Files/Source/DimGeopraphy.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Source/DimGeopraphy.csv".

display(df.limit(5))

StatementMeta(, 0050a1b7-1a65-4ae9-ace5-9e601e01c049, 22, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7ed20608-c123-4d4d-b319-deca163e5993)

## สร้าง dataframe ใหม่ชื่อ geo_df จาก dataframe เดิม

### โดยลบคอลัมน์ที่ไม่ใช้ออก

In [21]:
cols = ("SpanishCountryRegionName","FrenchCountryRegionName","PostalCode","SalesTerritoryKey","IpAddressLocator")

geo_df = df.drop(*cols)

display(geo_df.limit(5))

StatementMeta(, 0050a1b7-1a65-4ae9-ace5-9e601e01c049, 23, Finished, Available)

SynapseWidget(Synapse.DataFrame, c2ef93e0-5a60-4e15-950f-90a61e0fe23e)

## Join dataframe reseller_df กับ geo_df

In [22]:
join_df = reseller_df.join(geo_df,reseller_df.GeographyKey ==  geo_df.GeographyKey,"inner").drop(*["GeographyKey"])


display(join_df.limit(5))


StatementMeta(, 0050a1b7-1a65-4ae9-ace5-9e601e01c049, 24, Finished, Available)

SynapseWidget(Synapse.DataFrame, b7e9130b-ae01-4976-9cbc-91ff0a4a5362)

## เปลี่ยนชื่อคอลัมน์ และ เปลี่ยนชนิดข้อมูล

In [23]:
from pyspark.sql.functions import col,cast

rename_df = join_df.withColumnRenamed("StateProvinceCode","ProvinceCode").withColumnRenamed("StateProvinceName","Province")\
    .withColumnRenamed("CountryRegionCode","CountryCode").withColumnRenamed("EnglishCountryRegionName","Country")

 
output_df = rename_df.withColumn("ResellerKey", col("ResellerKey").cast("Integer"))

display(output_df.limit(5))

StatementMeta(, 0050a1b7-1a65-4ae9-ace5-9e601e01c049, 25, Finished, Available)

SynapseWidget(Synapse.DataFrame, 25b8838e-3a27-4e9e-a9cb-737563b1cfd7)

## เขียนลง Delta Table แบบ Managed Table

In [24]:
output_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("manage_DimReseller")


StatementMeta(, 0050a1b7-1a65-4ae9-ace5-9e601e01c049, 26, Finished, Available)

## เขียนลง Delta Table แบบ External

In [25]:
output_df.write.format("delta").saveAsTable("external_DimReseller", path="Files/external/external_DimReseller")

StatementMeta(, 0050a1b7-1a65-4ae9-ace5-9e601e01c049, 27, Finished, Available)